# Lecture 2

## 2.1 Jupyter notebooks

In [ ]:
import sys
print(sys.executable)
!which python3

In [ ]:
!pip --version
%pip --version

In [ ]:
%pip install flask
import flask

In [ ]:
import site
print(site.getsitepackages())
!ls -l {site.getsitepackages()[0]}

In [ ]:
%lsmagic

## 2.3 API requests using `requests`

In [ ]:
%pip install flask

In [ ]:
from flask import Flask, request, jsonify
import threading

app = Flask(__name__)

tasks = {}
i = 1

@app.post("/tasks")
def create():
    global i
    if not request.json: abort(400)
    t = {"id": i, "state": "created", "data": request.json}
    tasks[i] = t
    i += 1
    return jsonify(t), 201

# This starts Flask's blocking event loop in same thread as Jupyter
# Subsequent cells can't run until Flask stops serving
#app.run()

# Inside a Jupyter notebook then, run Flask in a background process
# `use_reloader = False` is mandatory in a Jupyter notebook
def run():
    app.run(host="127.0.0.1", port=5000, use_reloader=False)
threading.Thread(target=run, daemon=True).start()

In [ ]:
# Basic test using curl
!curl -X POST http://127.0.0.1:5000/tasks -H "Content-Type: application/json" -d '{"type": "demo", "params": {"x": 1}}'

In [ ]:
import requests
r = requests.post(
    "http://127.0.0.1:5000/tasks",
    json={"type": "demo", "params": {"x": 1}}
)
print(r.status_code, r.json())

In [ ]:
from flask import Flask, jsonify, abort, request
import threading

app = Flask(__name__)

tasks = {}

i = 1

@app.post("/tasks")
def create():
    global i
    if not request.json: abort(400)
    t = {"id": i, "state": "created", "data": request.json}
    tasks[i] = t
    i += 1
    return jsonify(t), 201

@app.get("/tasks")
def list_tasks():
    return jsonify(list(tasks.values()))

@app.get("/tasks/<int:i>")
def get_task(i):
    return jsonify(tasks[i]) if i in tasks else abort(404)

In [ ]:
def run():
    app.run(host="127.0.0.1", port=5000, use_reloader=False)
threading.Thread(target=run, daemon=True).start()

In [ ]:
import requests
r = requests.get("http://127.0.0.1:5000/tasks")
print(r.status_code, r.json())

In [ ]:
import requests
r = requests.post(
    "http://127.0.0.1:5000/tasks",
    json={"type": "demo", "params": {"x": 1}}
)
print(r.status_code, r.json())

In [ ]:
requests.post(
    "http://127.0.0.1:5000/tasks",
    json={"type": "demo", "params": {"x": 1}}
)

In [ ]:
requests.post(
    "http://127.0.0.1:5000/tasks",
    json={"type": "demo", "params": {"x": 1}}
).status_code

In [ ]:
requests.get("http://127.0.0.1:5000/tasks").json()

In [ ]:
requests.get("http://127.0.0.1:5000/tasks/2").json()

# Lecture 3

## 3.2 ecCodes

In [ ]:
import eccodes

In [ ]:
!grib_ls -V

In [ ]:
print(eccodes.codes_get_api_version())

### Read GRIB2 file

In [ ]:
!find .. -name "*.grib2"

In [ ]:
grib_file = "../e-ai_ml2/course/code/code03/ifs_2t.grib2"

In [ ]:
with open(grib_file, "rb") as f:
    while True:
        gid = eccodes.codes_grib_new_from_file(f)
        if gid is None: break

        short = eccodes.codes_get(gid, "shortName")
        level = eccodes.codes_get(gid, "level")
        size  = eccodes.codes_get_size(gid, "values")

        print(short, level, size)

        eccodes.codes_release(gid)

### Download GRIB2 file from ECMWF

In [ ]:
from ecmwf.opendata import Client

client = Client(
    source = "ecmwf",
    model = "ifs",
)

client.retrieve(
    time = 0,
    type = "fc",
    step = 24,
    param = ["2t", "msl"],
    target = "ifs_2t.grib2"
)

In [ ]:
!ls *.grib2

### Download from DWD

In [ ]:
import datetime

base_url = "http://opendata.dwd.de/weather/nwp/icon/grib/00/t_2m/"
now = datetime.datetime.now(datetime.UTC)
filename = f"icon_global_icosahedral_single-level_{now:%Y%m%d}00_000_T_2M.grib2.bz2"
url = base_url + filename
grib_filename = filename[:-4]

In [ ]:
import wget
wget.download(url, filename)

In [ ]:
import bz2

with bz2.open(filename, "rb") as f_in, open(grib_filename, "wb") as f_out:
    f_out.write(f_in.read())

In [ ]:
!ls *.grib2*

In [ ]:
import eccodes
with open(grib_filename, "rb") as f:
    while True:
        gid = eccodes.codes_grib_new_from_file(f)
        if gid is None: break

        short = eccodes.codes_get(gid, "shortName")
        level = eccodes.codes_get(gid, "level")
        size  = eccodes.codes_get_size(gid, "values")

        print(short, level, size)

        eccodes.codes_release(gid)

Extract and list metadata keys from a GRIB file:

In [ ]:
import eccodes
with open(grib_filename, "rb") as f:
    while True:
        gid = eccodes.codes_grib_new_from_file(f)
        if gid is None: break

        key_iterator = eccodes.codes_keys_iterator_new(gid)
        keys = []

        while eccodes.codes_keys_iterator_next(key_iterator):
            keyname = eccodes.codes_keys_iterator_get_name(key_iterator)
            if keyname not in ['section2Padding', 'codedValues', 'values']:
                value = eccodes.codes_get_string(gid, keyname)
            keys.append((keyname, value))

        eccodes.codes_release(gid)

        for key, value in keys:
              print(f"Key: {key:40} Value: {value}")

In [ ]:
import eccodes
with open(grib_file, "rb") as f:
    while True:
        gid = eccodes.codes_grib_new_from_file(f)
        if gid is None: break

        key_iterator = eccodes.codes_keys_iterator_new(gid)
        keys = []

        while eccodes.codes_keys_iterator_next(key_iterator):
            keyname = eccodes.codes_keys_iterator_get_name(key_iterator)
            if keyname not in ['section2Padding', 'codedValues', 'values']:
                value = eccodes.codes_get_string(gid, keyname)
            keys.append((keyname, value))

        eccodes.codes_release(gid)

        for key, value in keys:
              print(f"Key: {key:40} Value: {value}")

In [ ]:
%pip install cartopy
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

In [ ]:
with open(grib_file, "rb") as f:
    # First message is pressure
    gid = eccodes.codes_grib_new_from_file(f)

nx = eccodes.codes_get(gid, "Ni")
ny = eccodes.codes_get(gid, "Nj")
values = eccodes.codes_get_array(gid, "values")
field = values.reshape(ny, nx)

plt.figure(figsize=(7, 3.5))
plt.imshow(field)

In [ ]:
with open(grib_file, "rb") as f:
    # Run twice to get the second message (T2m)
    gid = eccodes.codes_grib_new_from_file(f)
    gid = eccodes.codes_grib_new_from_file(f)

nx = eccodes.codes_get(gid, "Ni")
ny = eccodes.codes_get(gid, "Nj")
values = eccodes.codes_get_array(gid, "values")
field = values.reshape(ny, nx)

plt.figure(figsize=(7, 3.5))
plt.imshow(field)

In [ ]:
fig, ax = plt.subplots(figsize=(10,5), subplot_kw={"projection": ccrs.PlateCarree()})
ax.coastlines()
ax.add_feature(cfeature.BORDERS)

lats   = eccodes.codes_get_array(gid, "latitudes")
lons   = eccodes.codes_get_array(gid, "longitudes")
lat   = lats.reshape(ny, nx)
lon   = lons.reshape(ny, nx)

ax.pcolormesh(lon, lat, field, transform=ccrs.PlateCarree(), cmap="jet")

In [ ]:
%pip install scipy
from scipy.interpolate import griddata


def load_grib(file, var):
    """Loads specified variable from GRIB file."""
    with open(file, 'rb') as f:
        while (gid := eccodes.codes_grib_new_from_file(f)) is not None:
            if eccodes.codes_get(gid, "shortName") == var:
                vals = eccodes.codes_get_array(gid, "values")
                eccodes.codes_release(gid)
                return vals
            eccodes.codes_release(gid)
    return None

def interpolate_to_grid(lat, lon, t2m, bbox, grid_res=0.25):
    """Interpolates T2M data onto a regular lat/lon grid."""
    latmin, latmax, lonmin, lonmax = bbox

    # Define a smooth regular grid
    grid_lat = np.arange(latmin, latmax, grid_res)
    grid_lon = np.arange(lonmin, lonmax, grid_res)
    lon_grid, lat_grid = np.meshgrid(grid_lon, grid_lat)

    points = np.column_stack((lon.ravel(), lat.ravel()))
    values = t2m.ravel()
    xi = np.column_stack((lon_grid.ravel(), lat_grid.ravel()))
    t2m_grid = griddata(points, values, xi, method='cubic')
    t2m_grid = t2m_grid.reshape(lon_grid.shape)
    
    return lon_grid, lat_grid, t2m_grid


def plot_t2m_grid(lat, lon, t2m, bbox, title, fname):
    """Plots interpolated 2m temperature as a smooth heatmap."""
    lon_grid, lat_grid, t2m_grid = interpolate_to_grid(lat, lon, t2m, bbox)

    # Set reasonable aspect ratio based on bounding box size
    lon_range = bbox[3] - bbox[2]
    lat_range = bbox[1] - bbox[0]
    aspect_ratio = lon_range / lat_range
    figsize = (10, max(5, 10 / aspect_ratio))  # Maintain consistent width & prevent extreme height

    plt.figure(figsize=figsize)
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.set_extent([bbox[2], bbox[3], bbox[0], bbox[1]])
    ax.add_feature(cfeature.LAND, edgecolor='black')
    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.BORDERS, linestyle=':')

    # Use smooth interpolation and correct aspect ratio
    img = ax.imshow(t2m_grid, extent=[bbox[2], bbox[3], bbox[0], bbox[1]], origin='lower',
                    cmap='jet', transform=ccrs.PlateCarree(), aspect='auto', interpolation='bicubic')

    plt.colorbar(img, label="Temperature (K)")
    plt.title(title)
    plt.savefig(fname, dpi=200, bbox_inches='tight')  # Reduce DPI for smaller file size
    plt.show()

    
# Load data
lat = load_grib("../e-ai_ml2/course/code/code03/icon_lat.grib", "tlat")
lon = load_grib("../e-ai_ml2/course/code/code03/icon_lon.grib", "tlon")
t2m = load_grib("../e-ai_ml2/course/code/code03/icon_t2m.grib", "2t")

# Plot interpolated global and Germany views
plot_t2m_grid(lat, lon, t2m, (-90, 90, -180, 180), "Interpolated Global 2m Temperature", "icon_t2m_global_interp.png")
plot_t2m_grid(lat, lon, t2m, (47, 55, 5, 15), "Interpolated 2m Temperature over Germany", "icon_t2m_germany_interp.png")

## 3.3 Accessing SYNOP observation files from NetCDF

In [ ]:
!find ../e-ai_ml2 -name "*.nc"

In [ ]:
%pip install netCDF4
from netCDF4 import Dataset

In [ ]:
import numpy as np

filename = "../e-ai_ml2/course/code/code03/synop.nc"

ncfile = Dataset(filename, "r")

lats = ncfile.variables["MLAH"][:]
lons = ncfile.variables["MLOH"][:]
temps = ncfile.variables["MTDBT"][:]

lats = np.array(lats)
lons = np.array(lons)
temps = np.array(temps)

ncfile.close()

In [ ]:
threshold=1e+20

import cartopy.crs as ccrs
#projections = [[ccrs.PlateCarree(), "PlateCarree"]]
projections=[[ccrs.PlateCarree(), "PlateCarree"], 
                                  [ccrs.TransverseMercator(), "TransverseMercator"],
                                  [ccrs.Mercator(), "Mercator"],
                                  [ccrs.EuroPP(), "EuroPP"],
                                  [ccrs.Geostationary(), "Geostationary"],
                                  [ccrs.Stereographic(), "Stereographic"]]
# Filter out large missing values
valid_mask = (temps < threshold) & np.isfinite(temps)
lats, lons, temps = lats[valid_mask], lons[valid_mask], temps[valid_mask]

import cartopy.feature as cfeature

for projection in projections:
        fig, ax = plt.subplots(figsize=(10, 6), subplot_kw={'projection': projection[0]})
        scatter = ax.scatter(lons, lats, c=temps, cmap='jet', s=5, alpha=0.7, transform=ccrs.PlateCarree())

        # Add map features
        ax.coastlines()
        ax.add_feature(cfeature.BORDERS, edgecolor='gray')
        ax.gridlines(draw_labels=True, linewidth=0.5, color='gray', alpha=0.5, linestyle='--')

        # Add colorbar with better spacing
        cbar = plt.colorbar(scatter, ax=ax, fraction=0.04, pad=0.08)  
        cbar.set_label("Temperature (K)")

        # Set title
        plt.title("Temperature Observations on Map in Projection " + projection[1])

        # Save and show the plot
        plt.show()

## 3.4 AIREP feedback files in NetCDF

In [ ]:
airep_file = "../e-ai_ml2/course/code/code03/monAIREP.nc"

ncfile = Dataset(airep_file, "r")

nc = 1
for varname in ncfile.variables.keys():
    var = ncfile.variables[varname]
    description = getattr(var, "longname", "N/A")
    dims = [len(ncfile.dimensions[dim]) for dim in var.dimensions]
    shape1 = dims[0] if len (dims) > 0 else ""
    shape2 = dims[1] if len (dims) > 1 else ""
    print ("{:<4} {:40} {:>10} {:>10} {:30}".format(nc, varname, shape1, shape2, description))
    if nc % 10 == 0:
        print("-" * 110)
    nc += 1

ncfile.close()

In [ ]:
# Read header-level variables
#ncfile = Dataset(airep_file, "r")
lat = ncfile.variables["lat"][:]
lon = ncfile.variables["lon"][:]

# Body-level variables
varno_all = ncfile.variables["varno"][:]
obs_all = ncfile.variables["obs"][:]
l_body = ncfile.variables["l_body"][:]

# Expand lat/lon to match body-level observations
ni = len(l_body)
ie = np.repeat(range(0, ni), l_body)  # Map each body entry to its header index

# varno == 2 is upper air temperature
idx = np.where(varno_all == 2)[0]

# Filter lat, lon, obs
lat_filtered = lat[ie[idx]]
lon_filtered = lon[ie[idx]]
obs_filtered = obs_all[idx]

var = "level"
var_data = ncfile.variables[var][:]

print(var_data.shape[0], len(varno_all))

extra_data = var_data[idx]
lats, lons, obs = lat_filtered, lon_filtered, obs_filtered
heights = extra_data

threshold=1e+20

print(len(lats), "Latitudes:", lats[:5])
print(len(lons), "Longitudes:", lons[:5])
print(len(obs), "Observations:", obs[:5])
if heights is not None:
    print(len(heights), "Heights:", heights[:5])

valid_mask = (obs < threshold) & np.isfinite(obs)
lats, lons, obs = lats[valid_mask], lons[valid_mask], obs[valid_mask]

# Keep only temperatures between -30°C and 40°C (243.15K to 313.15K)
temp_min, temp_max = 180, 320
physical_mask = (obs >= temp_min) & (obs <= temp_max)

lats_filtered, lons_filtered, obs_filtered = lats[physical_mask], lons[physical_mask], obs[physical_mask]

fig, ax = plt.subplots(figsize=(10, 6), subplot_kw={'projection': ccrs.PlateCarree()})

scatter = ax.scatter(lons_filtered, lats_filtered, c=obs_filtered, cmap='jet', s=2, alpha=0.7, transform=ccrs.PlateCarree())

ax.coastlines()
ax.add_feature(cfeature.BORDERS, edgecolor='gray')
ax.gridlines(draw_labels=True, linewidth=0.5, color='gray', alpha=0.5, linestyle='--')

# Ensure the colorbar does not exceed figure height
cbar = fig.colorbar(scatter, ax=ax, orientation='vertical', fraction=0.04, pad=0.08, shrink=0.8)
cbar.set_label("Temperature (K)")

plt.title("AIREP Observations")
plt.show()

## GPU access in practice

In [ ]:
import torch

In [ ]:
print(torch.cuda.is_available())

In [ ]:
print(torch.backends.mps.is_available())

In [ ]:
import time

In [ ]:
d = torch.device("mps")

In [ ]:
x = torch.rand((40000,40000),device=d)

In [ ]:
t0 = time.time()
y = torch.matmul(x, x)
torch.mps.synchronize()
print("Time = ", round(time.time()-t0,3))

In [ ]:
n = 40000
x0 = torch.rand((n, n), device="cpu")
x1 = torch.rand((n, n), device="cpu")
t0 = time.time()
y = torch.matmul(x0, x1)
print("Time = ", round(time.time() - t0, 3))

### Mixed precision

In [ ]:
%pip install wget

In [ ]:
import wget

In [ ]:
%pip --version

In [ ]:
!pip --version

## AI and ML

### Torch tensors

In [ ]:
import torch
x = torch.tensor([2., 3.], requires_grad=True)
y = x[0]**2 + x[1]**2
y.backward()
print(x.grad)

In [ ]:
import torch.nn as nn

# nn.Module is the base class for models and layers
# Holds parameters (weights and biases)
class SimpleNN(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Layer 1: 1 -> 16
        self.fc1 = nn.Linear(1,16)
        
        # Non-linear activation function (ReLU in this case)
        self.relu = nn.reLU()
        
        # Layer 2: 16 -> 1
        self.fc2 = nn.Linear(16,1)

    # Calling `model(x)` runs the model's `forward()` method
    # Forward pass computes predictions from inputs (x)
    # Builds the autograd graph (if grads enables on x)
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        return self.fc2(x)

Learning a sine function

In [ ]:
# Sample input values x
x = np.linspace(0, 2*np.pi, 1000)

# Compute labels y = sin(x)
y = np.sin(x)

plt.plot(x, y)
plt.show()

In [ ]:
# Dataset construction

from torch.utils.data import TensorDataset, DataLoader

x_t = torch.tensor(x).float().unsqueeze(1)
y_t = torch.tensor(y).float().unsqueeze(1)

data = TensorDataset(x_t, y_t)
loader = DataLoader(data,
                    batch_size=32,
                    shuffle=True)

In [ ]:
# Model and training loop

# Learn non-linear mapping x -> \hat{y}
# Input: scalar x
# Output: scalar \hat{y}

# Model
model = nn.Sequential(
    nn.Linear(1,16), nn.ReLU(),
    nn.Linear(16,16), nn.ReLU(),
    nn.Linear(16,1)
)

# Loss function
loss_fn = nn.MSELoss()

# Optimiser
opt = torch.optim.Adam(
    model.parameters(),
    lr = 0.01
)

# Training loop
#     - Compare \hat{y} and y
#     - Minimise prediction error
#     - Update model parameters
for x_b, y_b in loader:
    
    # Zero the gradients from the previous iteration
    opt.zero_grad()

    # Forward pass of the model to get predictions
    y_p = model(x_b)

    # Update loss given predictions y_p
    loss = loss_fn(y_p, y_b)

    # Backpropagation - compute gradients of loss wrt parameters
    loss.backward()

    # Optimiser - update parameters (weights and biases) in-place
    # given the gradients
    opt.step()